# Converter-Test: test of the Converters Module

In [14]:
import os
import sys
import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

RESULTS_PATH = Path('../choco/converters/validation/results')

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'choco'))

In [15]:
def open_results(results_path: str) -> None:
    pass

### Process results folder

In [88]:
converters_group = {
    'wikifonia': 'leadsheet',
    'ireal-pro': 'leadsheet',
    'weimar': 'leadsheet',
    'nottingham': 'leadsheet',
    'when-in-rome': 'roman',
    'rock-corpus': 'roman',
    'jazz-corpus': 'leadsheet',
    'band-in-a-box': 'prettify-harte',
    'mozart-piano-sonatas': 'roman',
}

biased_paths = [p for p in RESULTS_PATH.rglob('s2-0[1-2]/**/*.xlsx')]
unbiased_paths = [p for p in RESULTS_PATH.rglob('s2-0[3-4]/**/*.xlsx')]

print(f'Found {len(biased_paths)} biased evaluation files:\n {[x.stem for x in biased_paths]}\n')
print(f'Found {len(unbiased_paths)} unbiased evaluation files:\n {[x.stem for x in unbiased_paths]}\n')

Found 10 biased evaluation files:
 ['wikifonia_1194', 'wikifonia_1047', 'rock-corpus_4', 'rock-corpus_16', 'ireal-pro_914', 'ireal-pro_542', 'weimar_8', 'weimar_263', 'when-in-rome_109', 'when-in-rome_406']

Found 10 unbiased evaluation files:
 ['wikifonia_3442', 'wikifonia_4327', 'rock-corpus_34', 'rock-corpus_183', 'ireal-pro_558', 'ireal-pro_1103', 'weimar_433', 'weimar_279', 'when-in-rome_205', 'when-in-rome_61']



In [89]:
biased_validated = []

for biased_file in biased_paths:
    biased_data = pd.read_excel(biased_file, engine='openpyxl')
    validated_data = biased_data[biased_data['correct?'].notnull()]
    validated_data = validated_data.assign(partition=biased_file.stem.split('_')[0])
    validated_data = validated_data.assign(converter=converters_group[biased_file.stem.split('_')[0]])

    biased_validated.append(validated_data)

all_biased_validated = pd.concat(biased_validated)
all_biased_validated

,type,time,duration,original,converted,correct?,notes,partition,converter,Unnamed: 7
1,chord,4.0,3.0,B-7/E-,Bb:7/4,1.0,NaN,wikifonia,leadsheet,NaN
2,chord,8.0,2.0,E-,Eb:maj,1.0,NaN,wikifonia,leadsheet,NaN
3,chord,8.2,1.0,E-7,Eb:7,1.0,NaN,wikifonia,leadsheet,NaN
4,chord,9.0,3.0,A-/E-,Ab:maj/5,0.0,"I don't understand this one, Ab major always h...",wikifonia,leadsheet,NaN
5,chord,10.0,3.0,E-,Eb:maj,1.0,NaN,wikifonia,leadsheet,NaN
...,...,...,...,...,...,...,...,...,...,...
15,chord,13.1,2.0,C major:I64,G:maj/5,0.0,C maj / 5,when-in-rome,roman,NaN
16,chord,14.1,2.0,C major:V7,G:7,1.0,NaN,when-in-rome,roman,NaN
17,chord,15.1,2.0,C major:I64,G:maj/5,0.0,C maj / 5,when-in-rome,roman,NaN
18,chord,16.1,2.0,C major:V,G:maj,1.0,NaN,when-in-rome,roman,NaN


In [90]:
unbiased_validated = []

for unbiased_file in unbiased_paths:
    unbiased_data = pd.read_excel(unbiased_file, engine='openpyxl')
    unbiased_data = unbiased_data.assign(partition=unbiased_file.stem.split('_')[0])
    unbiased_data = unbiased_data.assign(converter=converters_group[unbiased_file.stem.split('_')[0]])

    unbiased_validated.append(unbiased_data)

all_unbiased_validated = pd.concat(unbiased_validated)
all_unbiased_validated

,original,converted,correct?,notes,partition,converter
0,D7,D:7,True,NaN,wikifonia,leadsheet
1,E-,Eb:maj,True,NaN,wikifonia,leadsheet
2,D7/F#,D:7/3,True,NaN,wikifonia,leadsheet
3,B-,Bb:maj,True,NaN,wikifonia,leadsheet
4,D7+,D:aug(b7),True,NaN,wikifonia,leadsheet
...,...,...,...,...,...,...
5,C major:iiø7,D:hdim7,True,NaN,when-in-rome,roman
6,C major:III+6/ii,F:aug/3,True,NaN,when-in-rome,roman
7,C major:I43,C:maj7/5,True,NaN,when-in-rome,roman
8,C major:IV64,F:maj/5,True,NaN,when-in-rome,roman


## Compute data

In [97]:
for x in all_biased_validated.groupby(['converter']):
    print('biased ', x[0], ' has accuracy: ', x[1]['correct?'].mean().round(2))

for x in all_unbiased_validated.groupby(['converter']):
    print('unbiased ', x[0], ' has accuracy: ', x[1]['correct?'].mean().round(2))


biased  leadsheet  has accuracy:  0.8
biased  roman  has accuracy:  0.75
unbiased  leadsheet  has accuracy:  0.91
unbiased  roman  has accuracy:  0.98
